In [3]:
###############################################################################
# This notebook contains a collection of useful tools used with SOE Assessment#
# Current list of tools:                                                      #
#  * List indicators with less then 4 items (or not multiple of 4 items)      #
#  * Bin tool (show equal width bins vs custom SOE bins)                      #
###############################################################################
# Core stuff
import os
from pathlib import Path
import re

# Data stuff
import pandas as pd # Data analysis
import numpy as np

# Pretty printing stuff
from IPython.display import display, HTML
import pprint
pp = pprint.PrettyPrinter(indent=4)

# Initial setup
country = 'RMI' # FSM
test = 'MISAT' # NMCT
#achievement_levels = ['well below competent', 'approaching competent', 'minimally competent', 'competent'] # NMCT
achievement_levels = ['Beginning', 'Developing', 'Proficient', 'Advanced'] # MISAT

In [4]:
def load_excel_to_df(filename):
    """Loads an Excel filename to a Pandas DataFrame.

    Parameters
    ----------
    filename : str, required
        The filename of the excel file to load

    Raises
    ------
    NotImplementedError
        Could raise unknown error. Implement if it happens
    
    Returns
    -------
    DataFrame
    """
    file_path = Path(filename)
    file_extension = file_path.suffix.lower()[1:]

    if file_extension == 'xlsx':
        df = pd.read_excel(filename, index_col=None, header=0, engine='openpyxl')
    elif file_extension == 'xls':
        df = pd.read_excel(filename, index_col=None, header=0)
    elif file_extension == 'csv':
        df = pd.read_csv(filename, index_col=None, header=0)
    else:
        raise Exception("File not supported")

    return df

In [5]:
###############################################################################
# Responses Sheet                                                             #
###############################################################################

# Load a single SOE Assessment workbook (for testing,)
# in particular the sheet with the raw data
cwd = os.getcwd()
#filename = os.path.join(cwd, 'data/RMI/MISAT/MISAT 2019/3GrEng2019/AllSchools_A03_2018-19_Results.xls')
#filename = os.path.join(cwd, 'data/RMI/MISAT/MISAT 2012/6grEng12/AllSchools_A06_2011-12_Results.xls')
#filename = os.path.join(cwd, 'data/RMI/MISAT/MISAT 2009/3GrEng09/AllSchools_A03_2008-09_Results.xls')
filename = os.path.join(cwd, 'data/RMI/MISAT/MISAT 2012/6GrEng2012/AllSchools_A06_2011-12_Results.xls')

df_student_results = load_excel_to_df(filename)
print('df_student_results')
display(df_student_results)

df_student_results


,RecordNo,SchoolYear,TestID,TestName,IslandName,SchoolID,SchoolName,StudentID,StudentName,Gender,...,Item_031_AS0602020603m_ccc,Item_032_AS0602020604h_ccc,Item_033_AS0602020101e_ddd,Item_034_AS0602020102m_ddd,Item_035_AS0602020103m_aaa,Item_036_AS0602020104h_bbb,Item_037_AS0602020401e_ccc,Item_038_AS0602020402m_ddd,Item_039_AS0602020403m_bbb,Item_040_AS0602020404h_ccc
0,1,2011-12,A06,A06 - Reading Grade 6 - English,Aelonlaplap,101,Airok A,419,Jally Kedibad,M,...,A,D,A,B,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2011-12,A06,A06 - Reading Grade 6 - English,Aelonlaplap,101,Airok A,420,Macklynn Kedibad,F,...,D,D,B,A,B,D,B,C,NaN,NaN
2,3,2011-12,A06,A06 - Reading Grade 6 - English,Aelonlaplap,101,Airok A,421,Sign Henson,F,...,A,C,D,B,B,A,C,C,B,C
3,4,2011-12,A06,A06 - Reading Grade 6 - English,Aelonlaplap,101,Airok A,422,Wadik Samuel,F,...,B,A,C,B,D,A,C,B,D,C
4,5,2011-12,A06,A06 - Reading Grade 6 - English,Aelonlaplap,102,Buoj,423,Jimor Samule,M,...,B,A,D,C,D,B,B,D,B,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,1163,2011-12,A06,A06 - Reading Grade 6 - English,Wotje,194,Wotje,496,Kano Lanwi,M,...,D,B,D,C,A,B,B,A,D,C
1163,1164,2011-12,A06,A06 - Reading Grade 6 - English,Wotto,195,Wotto,99,Monica Ainrik,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1164,1165,2011-12,A06,A06 - Reading Grade 6 - English,Wotto,195,Wotto,100,Junior Botlok,M,...,A,C,D,C,B,A,D,A,C,D
1165,1166,2011-12,A06,A06 - Reading Grade 6 - English,Wotto,195,Wotto,101,Aaron Elanzo,M,...,A,C,C,B,C,A,B,A,C,B


In [5]:
%%time
###############################################################################
# Responses Sheet (all)                                                       #
###############################################################################

# Load all SOE Assessment workbook inside a directory
# (~50 seconds on iMac with i9 CPU and 32GB RAM)
cwd = os.getcwd()
path = os.path.join(cwd, 'data/'+country+'/'+test+'/')

df_student_results_list = []

for root, directories, files in os.walk(path, topdown=False):
    for name in files:
        filename = os.path.join(root, name)
        print('Loading into DataFrame:', filename)
        try:
            df_student_results_list.append(load_excel_to_df(filename))
        except:
            print('Problem loading:', filename)
            #print('Error was:', )            

print('Completed loading excel files')

Loading into DataFrame: /mnt/c/Users/Ghislain Hachey/Google Drive (ghachey@nuzusys.com)/Development/Pacific EMIS/repositories/pacific-emis-exams-data-jupyter-python/data/RMI/MISAT/MISAT 2009/3GrEng2009/AllSchools_A03_2008-09_Results.xls
Loading into DataFrame: /mnt/c/Users/Ghislain Hachey/Google Drive (ghachey@nuzusys.com)/Development/Pacific EMIS/repositories/pacific-emis-exams-data-jupyter-python/data/RMI/MISAT/MISAT 2009/3GrKM2009/AllSchools_B03_2008-09_Results.xls
Loading into DataFrame: /mnt/c/Users/Ghislain Hachey/Google Drive (ghachey@nuzusys.com)/Development/Pacific EMIS/repositories/pacific-emis-exams-data-jupyter-python/data/RMI/MISAT/MISAT 2009/3GrMath2009/AllSchools_M03_2008-09_Results.xls
Loading into DataFrame: /mnt/c/Users/Ghislain Hachey/Google Drive (ghachey@nuzusys.com)/Development/Pacific EMIS/repositories/pacific-emis-exams-data-jupyter-python/data/RMI/MISAT/MISAT 2009/6GrEng2009/AllSchools_A06_2008-09_Results.xls
Loading into DataFrame: /mnt/c/Users/Ghislain Hachey

In [6]:
%%time
# e.g. {'A.6.2.1.4': ['Item_001_AS0602010401E_ddd', 'Item_002_AS0602010402M_aaa', etc.]}
indicators_items = {}
# e.g. {'A.6.2.1': ['Item_001_AS0602010401E_ddd', 'Item_002_AS0602010402M_aaa', etc.]}
benchmarks_items = {}
# e.g. {'A.6.2': ['Item_001_AS0602010401E_ddd', 'Item_002_AS0602010402M_aaa', etc.]}
standards_items = {}

def compile_indicators(item, test='N/A', year='N/A'):
    """ A function to compile the related items into their indicators (e.g. Test.Grade.Standard.Benchmark.Indicator).
    Note - In addition, this function also compiles the benchmarks_items ina similar way.

    Parameters
    ----------
    item : String, required
        The item string (e.g. Item_002_AS0602010402M_aaa)
        
    Raises
    ------
    NotImplementedError
        Could raise unknown error. Implement if it happens
    
    Returns
    -------
    Nothing
    """
    item_meta = item.split('_')
    item_parts = list(item_meta[2])
    indicator = (test, year, item_parts[0] + '.' + item_parts[3] + '.' + item_parts[5] + '.' + item_parts[7] + '.' + item_parts[9])
    benchmark = (test, year, item_parts[0] + '.' + item_parts[3] + '.' + item_parts[5] + '.' + item_parts[7])
    standard = (test, year, item_parts[0] + '.' + item_parts[3] + '.' + item_parts[5])
    
    # Check if indicator already added, if not add it
    if indicator in indicators_items:       
        indicators_items[indicator].append(item)
    else:
        indicators_items[indicator] = [item]
        
    # Check if benchmark already added, if not add it
    if benchmark in benchmarks_items:       
        benchmarks_items[benchmark].append(item)
    else:
        benchmarks_items[benchmark] = [item]    
        
    # Check if standard already added, if not add it
    if standard in standards_items:       
        standards_items[standard].append(item)
    else:
        standards_items[standard] = [item]        

for df in df_student_results_list:
    test = df['TestID'][0]
    year = df['SchoolYear'][0]
    cols = df.columns.values
    cols_items = [i for i in cols if 'Item_' in i]

    for i in cols_items:
        compile_indicators(i, test, year)
 
# output too long with all SOE workbooks processed
#print('Indicators items')
#print('----------------')
#pp.pprint(indicators_items)
#print('Benchmarks items')
#print('----------------')
#pp.pprint(benchmarks_items)
#print('Standards items')
#print('----------------')
#pp.pprint(standards_items)

CPU times: user 19.3 ms, sys: 10 ms, total: 29.3 ms
Wall time: 28.1 ms


In [7]:
# Cycle through the indicators items and count/identify the ones that
#  * have less than 4 items
#  * do not have a number of item that is a multiple of 4 (4, 8, 12, etc.) items

indicators_without_items = []
indicators_less_than_four = []
indicators_not_multiple_of_four = []

for indicator in indicators_items:
    #print(len(indicators_items[indicator]))
    if (len(indicators_items[indicator]) == 0):
        indicators_without_items.append(indicator)    
    elif (len(indicators_items[indicator]) < 4):
        indicators_less_than_four.append((indicator, str(len(indicators_items[indicator]))+' items'))
    elif (len(indicators_items[indicator]) % 4 != 0):
        indicators_not_multiple_of_four.append((indicator, str(len(indicators_items[indicator]))+' items'))

print('Number of Indicators with less than 4 items')
print('-------------------------------------------')
pp.pprint(indicators_less_than_four)
print(len(indicators_less_than_four))
print('Number of Indicators with number of items that is not a multiple of 4')
print('---------------------------------------------------------------------')
pp.pprint(indicators_not_multiple_of_four)
print(len(indicators_not_multiple_of_four))
print('Number of Indicators without any items')
print('--------------------------------------')
pp.pprint(indicators_without_items)
print(len(indicators_without_items))

Number of Indicators with less than 4 items
-------------------------------------------
[   (('A03', '2008-09', 'A.3.1.4.3'), '3 items'),
    (('B06', '2008-09', 'A.6.1.4.1'), '3 items'),
    (('H08', '2008-09', 'W.8.3.2.1'), '1 items'),
    (('M06', '2009-10', 'M.6.1.5.1'), '2 items'),
    (('M06', '2009-10', 'M.6.1.5.2'), '2 items'),
    (('M06', '2010-11', 'M.6.1.5.1'), '2 items'),
    (('M06', '2010-11', 'M.6.1.5.2'), '2 items'),
    (('M06', '2011-12', 'M.6.1.5.1'), '2 items'),
    (('M06', '2011-12', 'M.6.1.5.2'), '2 items'),
    (('M06', '2012-13', 'M.6.1.5.1'), '2 items'),
    (('M06', '2012-13', 'M.6.1.5.2'), '2 items'),
    (('M06', '2014-15', 'M.6.1.5.1'), '2 items'),
    (('M06', '2014-15', 'M.6.1.5.2'), '2 items'),
    (('M06', '2015-16', 'M.6.1.5.1'), '2 items'),
    (('M06', '2015-16', 'M.6.1.5.2'), '2 items'),
    (('H08', '2015-16', 'M.8.1.1.1'), '3 items'),
    (('H08', '2015-16', 'M.8.1.4.1'), '1 items'),
    (('H08', '2015-16', 'C.8.2.4.1'), '3 items'),
    (('E12',

In [8]:
# Cycle through the benchmark items and count/identify the ones that
#  * have less than 4 items
#  * do not have a number of item that is a multiple of 4 (4, 8, 12, etc.) items

benchmarks_without_items = []
benchmarks_less_than_four = []
benchmarks_not_multiple_of_four = []

for benchmark in benchmarks_items:
    #print(len(benchmarks_items[benchmark]))
    if (len(benchmarks_items[benchmark]) == 0):
        benchmarks_without_items.append(benchmark)    
    elif (len(benchmarks_items[benchmark]) < 4):
        benchmarks_less_than_four.append((benchmark, str(len(benchmarks_items[benchmark]))+' items'))
    elif (len(benchmarks_items[benchmark]) % 4 != 0):
        benchmarks_not_multiple_of_four.append((benchmark, str(len(benchmarks_items[benchmark]))+' items'))

print('Number of benchmarks with less than 4 items')
print('-------------------------------------------')
pp.pprint(benchmarks_less_than_four)
print(len(benchmarks_less_than_four))
print('Number of benchmarks with number of items that is not a multiple of 4')
print('---------------------------------------------------------------------')
pp.pprint(benchmarks_not_multiple_of_four)
print(len(benchmarks_not_multiple_of_four))
print('Number of benchmarks without any items')
print('--------------------------------------')
pp.pprint(benchmarks_without_items)
print(len(benchmarks_without_items))

Number of benchmarks with less than 4 items
-------------------------------------------
[   (('H08', '2008-09', 'W.8.3.2'), '1 items'),
    (('H08', '2015-16', 'M.8.1.1'), '3 items'),
    (('H08', '2015-16', 'M.8.1.4'), '1 items'),
    (('H08', '2015-16', 'C.8.2.4'), '3 items'),
    (('E12', '2016-17', 'E.2.3.2'), '3 items'),
    (('H08', '2016-17', 'M.8.1.1'), '3 items'),
    (('H08', '2016-17', 'M.8.1.4'), '1 items'),
    (('H08', '2016-17', 'C.8.2.4'), '3 items'),
    (('H08', '2018-19', 'M.8.1.1'), '3 items'),
    (('H08', '2018-19', 'M.8.1.4'), '1 items'),
    (('H08', '2018-19', 'C.8.2.4'), '3 items'),
    (('H08', '2019-20', 'W.8.3.2'), '2 items')]
12
Number of benchmarks with number of items that is not a multiple of 4
---------------------------------------------------------------------
[   (('B06', '2008-09', 'A.6.1.4'), '7 items'),
    (('H08', '2008-09', 'M.8.1.1'), '11 items'),
    (('H08', '2009-10', 'B.8.3.3'), '9 items'),
    (('H08', '2015-16', 'C.8.2.5'), '5 items'),


In [9]:
# Playing with Bin values into discrete intervals.

# Example with IntervalIndex. 
# I actually prefer using int (for equal bins) or sequence of scalar (for non-uniform width/edges)
#bins = pd.IntervalIndex.from_tuples([(0, 1), (2, 3), (4, 5)])
#display(bins)
#pd.cut([0, 0.5, 1.5, 2.5, 4.5], bins, retbins=True, include_lowest=True)

print('Shows for each total number of items correct what level would be achieved')
print('=========================================================================')
print()

s1 = pd.Series(np.array([0, 1]))
c1a = pd.cut(s1, 4, labels=achievement_levels, retbins=True, right=True) # Default
c1b = pd.cut(s1, [-0.002,  -0.001 ,  0.5  ,  0.75 ,  1. ], labels=achievement_levels, retbins=True, include_lowest=True) # SOE
print('Bins with 1 Item')
print('-----------------')
display('Equal width bins', c1a)
display('SOE equivalent bins', c1b)
print()

s2 = pd.Series(np.array([0, 1, 2]))
c2a = pd.cut(s2, 4, labels=achievement_levels, retbins=True, right=True)
c2b = pd.cut(s2, [-0.002,  -0.001  ,  0.999   ,  1.5  ,  2.], labels=achievement_levels, retbins=True, include_lowest=True)
print('Bins with 2 Items')
print('-----------------')
display('Equal width bins', c2a)
display('SOE equivalent bins', c2b)
print()

s3 = pd.Series(np.array([0, 1, 2, 3]))
c3a = pd.cut(s3, 4, labels=achievement_levels, retbins=True, right=True)
c3b = pd.cut(s3, [-0.003,  0.75 ,  1.5  ,  2.25 ,  3.], labels=achievement_levels, retbins=True, include_lowest=True)
print('Bins with 3 Items')
print('-----------------')
display('Equal width bins', c3a)
display('SOE equivalent bins', c3b)
print()

s4 = pd.Series(np.array([0, 1, 2, 3, 4]))
c4a = pd.cut(s4, 4, labels=achievement_levels, retbins=True, right=True)
#c4b = pd.cut(s4, [0,1,2,3,4], labels=achievement_levels, retbins=True, include_lowest=True)
print('Bins with 4 Items')
print('-----------------')
display('Equal width bins', c4a)
#display('SOE equivalent bins', c4b)
print()

s5 = pd.Series(np.array([0, 1, 2, 3, 4, 5]))
c5a = pd.cut(s5, 4, labels=achievement_levels, retbins=True, right=True)
c5b = pd.cut(s5, [-0.005,  1.25 ,  3.5  ,  4.75 ,  5.], labels=achievement_levels, retbins=True, right=True, include_lowest=True)
print('Bins with 5 Items')
print('-----------------')
display('Equal width bins', c5a)
display('SOE equivalent bins', c5b)
print()

s8 = pd.Series(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8]))
c8a = pd.cut(s8, 4, labels=achievement_levels, retbins=True, right=True)
#c8b = pd.cut(s8, [0,2,4,6,8], labels=achievement_levels, retbins=True, include_lowest=True)
print('Bins with 8 Items')
print('-----------------')
display('Equal width bins', c8a)
#display('SOE equivalent bins', c8b)
print()

s9 = pd.Series(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))
c9a = pd.cut(s9, 4, labels=achievement_levels, retbins=True, right=True)
c9b = pd.cut(s9, [-0.009,  4.25 ,  6.5  ,  7.75 ,  9. ], labels=achievement_levels, retbins=True, include_lowest=True)
print('Bins with 9 Items')
print('-----------------')
display('Equal width bins', c9a)
display('SOE equivalent bins', c9b)
print()

s11 = pd.Series(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]))
c11a = pd.cut(s11, 4, labels=achievement_levels, retbins=True, right=True)
c11b = pd.cut(s11, [-0.011,  5.75,  7.50,  9.25,  11.0], labels=achievement_levels, retbins=True, include_lowest=True)
print('Bins with 11 Items')
print('-----------------')
display('Equal width bins', c11a)
display('SOE equivalent bins', c11b)
print()

s12 = pd.Series(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]))
c12a = pd.cut(s12, 4, labels=achievement_levels, retbins=True, right=True)
#c12b = pd.cut(s12, [0,3,6,9,12], labels=achievement_levels, retbins=True, include_lowest=True)
print('Bins with 12 Items')
print('-----------------')
display('Equal width bins', c12a)
#display('SOE equivalent bins', c12b)
print()

Shows for each total number of items correct what level would be achieved

Bins with 1 Item
-----------------


'Equal width bins'

(0    Beginning
 1     Advanced
 dtype: category
 Categories (4, object): ['Beginning' < 'Developing' < 'Proficient' < 'Advanced'],
 array([-0.001,  0.25 ,  0.5  ,  0.75 ,  1.   ]))

'SOE equivalent bins'

(0    Developing
 1      Advanced
 dtype: category
 Categories (4, object): ['Beginning' < 'Developing' < 'Proficient' < 'Advanced'],
 array([-0.002, -0.001,  0.5  ,  0.75 ,  1.   ]))

...

'SOE equivalent bins'

(0      Beginning
 1      Beginning
 2      Beginning
 3      Beginning
 4      Beginning
 5      Beginning
 6     Developing
 7     Developing
 8     Proficient
 9     Proficient
 10      Advanced
 11      Advanced
 dtype: category
 Categories (4, object): ['Beginning' < 'Developing' < 'Proficient' < 'Advanced'],
 array([-1.10e-02,  5.75e+00,  7.50e+00,  9.25e+00,  1.10e+01]))


Bins with 12 Items
-----------------


'Equal width bins'

(0      Beginning
 1      Beginning
 2      Beginning
 3      Beginning
 4     Developing
 5     Developing
 6     Developing
 7     Proficient
 8     Proficient
 9     Proficient
 10      Advanced
 11      Advanced
 12      Advanced
 dtype: category
 Categories (4, object): ['Beginning' < 'Developing' < 'Proficient' < 'Advanced'],
 array([-0.012,  3.   ,  6.   ,  9.   , 12.   ]))